In [ ]:
import torch
print(torch.__version__)
#!pip install torchtext==0.5
#!pip install -U torch==1.5
#!pip install tokenizers
!pip install nbresuse

In [ ]:
#!ls ../data/ncs_preprocessed_data/train-ncs/
#!pip install tokenizers
#tokenizer.decode(tokenizer.encode("hello there").ids)

!git clone https://github.com/NVIDIA/apex
!cd apex

In [ ]:
#ls ../data/ncs_preprocessed_data/train-ncs/
!cd apex;pip install -v --no-cache-dir ./
#!print(src_tokenizer)

In [ ]:
print(src_tokenizer.encode('<s> </s> <pad>').ids)
print(src_tokenizer.encode(' hi there ').ids)
print(tgt_tokenizer.decode(src_tokenizer.encode(' hi there ').ids))


In [1]:
from apex import amp
from tokenizers import ByteLevelBPETokenizer
src_tokenizer = ByteLevelBPETokenizer()


src_tokenizer.train(["../data/ncs_preprocessed_data/train-ncs/code.original_subtoken"], vocab_size=30000, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"])

tgt_tokenizer = ByteLevelBPETokenizer()
tgt_tokenizer.train(["../data/ncs_preprocessed_data/train-ncs/javadoc.original"], vocab_size=20000, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"])

In [2]:
vars(src_tokenizer)

{'_tokenizer': <tokenizers.Tokenizer at 0x559c1a79cec0>,
 '_parameters': {'model': 'ByteLevelBPE',
  'add_prefix_space': False,
  'lowercase': False,
  'dropout': None,
  'unicode_normalizer': None,
  'continuing_subword_prefix': None,
  'end_of_word_suffix': None,
  'trim_offsets': False}}

In [3]:
print(src_tokenizer.encode('<s> </s> <pad> <unk>').ids) #[0, 225, 2, 225, 1]
print(src_tokenizer.encode(' hi there ').ids)

print(tgt_tokenizer.encode('<s> </s> <pad> <unk>').ids) #[0, 225, 2, 225, 1]
print(tgt_tokenizer.encode(' hi there ').ids)

print(src_tokenizer.decode([13000]))
print(src_tokenizer)
print(tgt_tokenizer)


[0, 225, 2, 225, 1, 225, 3]
[3584, 368, 1051, 225]
[0, 225, 2, 225, 1, 225, 3]
[326, 77, 748, 225]
 dic
Tokenizer(vocabulary_size=30000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)
Tokenizer(vocabulary_size=20000, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)


In [4]:
from torch.utils.data import Dataset, DataLoader
from joblib import Parallel, delayed
import threading
import linecache
linecache.clearcache()
import subprocess
import os
from tqdm import tqdm
import torch

MAX_SRC_LEN = 200
MAX_TGT_LEN = 50


def pad_sequences(x, max_len):
    padded = torch.ones((max_len), dtype=torch.long)
    if len(x) > max_len: padded[:] = torch.tensor(x[:max_len] , dtype=torch.long)
    else: padded[:len(x)] = torch.tensor(x, dtype=torch.long)
    return padded

class LazyDataset(Dataset):
    def __init__(self, src_tokenizer,tgt_tokenizer, src_path, tgt_path, max_len_src = MAX_SRC_LEN,max_len_tgt=MAX_TGT_LEN):
        self.src_path = src_path
        self.tgt_path = tgt_path
        self.max_len_src = max_len_src
        self.max_len_tgt = max_len_tgt
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer
        self.num_entries = sum(1 for line in open(src_path))
        self.target_entries = sum(1 for line in open(tgt_path))
        print(self.num_entries, self.target_entries)
        assert self.num_entries == self.target_entries
        
            
    def __getitem__(self, idx):
        x = self.src_tokenizer.encode("<s>"+linecache.getline(self.src_path, idx + 1).strip()+"</s>").ids
        y = self.tgt_tokenizer.encode("<s>"+linecache.getline(self.tgt_path, idx + 1).strip()+"</s>").ids
        
        return torch.tensor(pad_sequences(x,self.max_len_src), dtype=torch.long),torch.tensor(pad_sequences(y,self.max_len_tgt), dtype=torch.long) 
    
    def __len__(self):
        return self.num_entries

train_dataset = LazyDataset(src_tokenizer, tgt_tokenizer, '../data/ncs_preprocessed_data/train-ncs/code.original_subtoken',
                            '../data/ncs_preprocessed_data/train-ncs/javadoc.original')



69708 69708


In [5]:
test_dataset = LazyDataset(src_tokenizer, tgt_tokenizer, '../data/ncs_preprocessed_data/test/code.original_subtoken',
                            '../data/ncs_preprocessed_data/test/javadoc.original')


8714 8714


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [7]:
#!conda install spacy
#!python -m spacy.en.download
#!spacy download en
#spacy_en = spacy.load('en')

#!pip install spacy

In [8]:
'''
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]
'''

'\ndef tokenize_en(text):\n    """\n    Tokenizes English text from a string into a list of strings\n    """\n    return [tok.text for tok in spacy_en.tokenizer(text)]\n'

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = MAX_SRC_LEN):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, src len]
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention
    
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = MAX_TGT_LEN):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, trg len]
        #src_mask = [batch size, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention
    
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [10]:

INPUT_DIM = 30000
OUTPUT_DIM = 20000



HID_DIM = 256
ENC_LAYERS = 6
DEC_LAYERS = 6
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [11]:
model = Seq2Seq(enc, dec, 1, 1, device).to(device)

In [12]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,911,328 trainable parameters


In [13]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [14]:
model.apply(initialize_weights)
#LEARNING_RATE = 0.0005
LEARNING_RATE = 0.00005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = 1)


In [15]:
from tqdm import tqdm
def train(model, dataset, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    for (src_, trg_) in tqdm(dataset):
        
        optimizer.zero_grad()

        src, trg = src_.to(device), trg_.to(device)
        output, _ = model(src, trg[:,:-1])

        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
        #print(trg.shape)
        #print(output.shape)
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim) #[:, :-1, :]
        trg = trg[:,1:].contiguous().view(-1)

        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
        #print(trg.shape)
        #print(output.shape)
        loss = criterion(output, trg)
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        #print(f'\tTrain Loss: {loss.item():.3f} | Train PPL: {math.exp(loss.item()):7.3f}')

        
    return epoch_loss / len(dataset)

In [16]:
def evaluate(model,dataset,criterion):
    
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for (src_, trg_) in tqdm(dataset):
            src, trg = src_.to(device), trg_.to(device)
            output, _ = model(src, trg[:,:-1])

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim) #[:, :-1, :]
            trg = trg[:,1:].contiguous().view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(dataset)

def give_pred(model, tokens, device, max_len=50):
    model.eval()
    
    src_indexes = tokens
    #print(tokens.size())
    src_tensor = torch.LongTensor(src_indexes).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = torch.zeros((tokens.size()[0], 1), dtype = torch.long)
    trg_indexes = trg_indexes.tolist()
    #print(trg_indexes)
    #print(trg_indexes.size())
    
    for i in range(max_len):
        #print(len(trg_indexes),len(trg_indexes[0]))
        #trg_tensor = torch.LongTensor(pad_sequences(trg_indexes, 50)).unsqueeze(0).to(device)
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        #print("in loop", trg_tensor.size())
        trg_mask = model.make_trg_mask(trg_tensor)
        #print(trg_mask)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        #print("out shape", output.size())
        
        
        
        for idx in range(len(trg_indexes)):
            #print("shape here = ", output[i][-1].size())
            #print(output[idx][-1].argmax().item())
            trg_indexes[idx].append(output[idx][-1].argmax().item())
            
        
        #pred_token = output.argmax(2)[:,-1].item()
        #print(output[0][-1].argmax())
        #print(pred_token)
        #trg_indexes.append(pred_token)

        #if pred_token == 2:
        #    break
    #print(len(trg_indexes),len(trg_indexes[0]))
    return trg_indexes

def calculate_blue(model,test_ds, device, max_len = 50):
    model.eval()
    sum_blue = 0
    data_count = 0
    for (src_, trg_) in tqdm(test_ds):
        hypothesis = give_pred(model,src_, device, max_len = 50)
        reference = trg_.tolist()
        #there may be several references
        for i in range(src_.size()[0]):
            sum_blue += nltk.translate.bleu_score.sentence_bleu([reference[i]], hypothesis[i])
            data_count+=1
        #print(sum_blue/data_count)
        #print(data_count)
    return sum_blue/data_count

'''
import warnings
import nltk
warnings.filterwarnings('ignore')

N_EPOCHS = 20
CLIP = 1

BATCH_SIZE = 280
EVAL_BATCH_SIZE = 500

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=True)

test_dataset = DataLoader(test_dataset, batch_size = EVAL_BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=False)


calculate_blue(model,test_dataset, device, max_len = 50)
'''

"\nimport warnings\nimport nltk\nwarnings.filterwarnings('ignore')\n\nN_EPOCHS = 20\nCLIP = 1\n\nBATCH_SIZE = 280\nEVAL_BATCH_SIZE = 500\n\ntrain_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=5,\n                     drop_last=True,\n                     shuffle=True)\n\ntest_dataset = DataLoader(test_dataset, batch_size = EVAL_BATCH_SIZE, num_workers=5,\n                     drop_last=True,\n                     shuffle=False)\n\n\ncalculate_blue(model,test_dataset, device, max_len = 50)\n"

In [17]:
'''
for (src_, trg_) in test_dataset:
    for i in range(src_.size()[0]):
        print(src_.size())
        hypothesis = give_pred(model,src_[i], device, max_len = 50)
        reference = trg_[i].tolist()
        #there may be several references
        sum_blue = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
        #data_count+=1
'''

'\nfor (src_, trg_) in test_dataset:\n    for i in range(src_.size()[0]):\n        print(src_.size())\n        hypothesis = give_pred(model,src_[i], device, max_len = 50)\n        reference = trg_[i].tolist()\n        #there may be several references\n        sum_blue = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)\n        #data_count+=1\n'

In [ ]:
import warnings
import nltk
warnings.filterwarnings('ignore')

N_EPOCHS = 20
CLIP = 1

BATCH_SIZE = 220
EVAL_BATCH_SIZE = 400

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=True)

test_dataset = DataLoader(test_dataset, batch_size = EVAL_BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=False)


#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataset))
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)



def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

best_valid_loss = float('inf')

opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
    

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train(model, train_dataset, optimizer, criterion, CLIP)
    #scheduler.step()
    scheduler.step(train_loss)
    valid_loss = evaluate(model, test_dataset, criterion)
    
    blue_score = calculate_blue(model,test_dataset, device, max_len = 50)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("validation blue score = ",blue_score*100)
    '''
    annealing
    Epoch: 14 | Time: 1m 43s
    Train Loss: 4.060 | Train PPL:  57.988
    
    Epoch: 07 | Time: 1m 58s
	Train Loss: 4.715 | Train PPL: 111.645
	 Val. Loss: 4.773 |  Val. PPL: 118.230
    '''

  0%|          | 0/348 [00:00<?, ?it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


 34%|███▍      | 120/348 [00:47<01:27,  2.61it/s]

In [ ]:
train_dataset = LazyDataset(src_tokenizer, tgt_tokenizer, '../data/ncs_preprocessed_data/train-ncs/code.original_subtoken',
                            '../data/ncs_preprocessed_data/train-ncs/javadoc.original')


train_dataset = DataLoader(train_dataset, batch_size = 101, num_workers=10,
                     drop_last=True,
                     shuffle=True)
for (src_, trg_) in tqdm(train_dataset):
    src, trg = src_.to(device), trg_.to(device)
    print(src[0].size())

In [ ]:
#load and save here

# Save checkpoint
checkpoint = {
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'amp': amp.state_dict()
}
torch.save(checkpoint, '../models/big_checkpoint_corrected_600_loss1.pt')



In [ ]:

# Restore
opt_level = 'O1'
model = Seq2Seq(enc, dec, 1, 1, device).to(device)
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

checkpoint = torch.load('../models/big_checkpoint_corrected_600_loss1.pt')

model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
amp.load_state_dict(checkpoint['amp'])


import warnings
import nltk
warnings.filterwarnings('ignore')

N_EPOCHS = 20
CLIP = 1

BATCH_SIZE = 280
EVAL_BATCH_SIZE = 500

train_dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=True)

test_dataset = DataLoader(test_dataset, batch_size = EVAL_BATCH_SIZE, num_workers=5,
                     drop_last=True,
                     shuffle=False)


In [ ]:

def give_pred(model, tokens, device, max_len=50):
    model.eval()
    
    src_indexes = tokens
    #print(tokens.size())
    src_tensor = torch.LongTensor(src_indexes).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = torch.zeros((tokens.size()[0], 1), dtype = torch.long)
    trg_indexes = trg_indexes.tolist()
    #print(trg_indexes)
    #print(trg_indexes.size())
    
    for i in range(max_len):
        #print(len(trg_indexes),len(trg_indexes[0]))
        #trg_tensor = torch.LongTensor(pad_sequences(trg_indexes, 50)).unsqueeze(0).to(device)
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        #print("in loop", trg_tensor.size())
        trg_mask = model.make_trg_mask(trg_tensor)
        #print(trg_mask)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        #print("out shape", output.size())
        
        
        
        for idx in range(len(trg_indexes)):
            #print("shape here = ", output[i][-1].size())
            #print(output[idx][-1].argmax().item())
            trg_indexes[idx].append(output[idx][-1].argmax().item())
            
        
        #pred_token = output.argmax(2)[:,-1].item()
        #print(output[0][-1].argmax())
        #print(pred_token)
        #trg_indexes.append(pred_token)

        #if pred_token == 2:
        #    break
    #print(len(trg_indexes),len(trg_indexes[0]))
    return trg_indexes

def calculate_blue(model,test_ds, device, max_len = 50):
    model.eval()
    sum_blue = 0
    data_count = 0
    for (src_, trg_) in tqdm(test_ds):
        hypothesis = give_pred(model,src_, device, max_len = 50)
        reference = trg_.tolist()
        #there may be several references
        for i in range(src_.size()[0]):
            sum_blue += nltk.translate.bleu_score.sentence_bleu([reference[i]], hypothesis[i])
            data_count+=1
        print(sum_blue/data_count)
        print(data_count)
    return sum_blue/data_count

calculate_blue(model,test_dataset, device, max_len = 50)



In [ ]:

def translate_sentence(sentence, model, device, max_len = 50):    
    model.eval()
        # add <s> </s>
    #    "<s> "+linecache.getline(self.src_path, idx + 1).strip()+" </s>"
    src_indexes = pad_sequences(src_tokenizer.encode("<s>"+sentence.strip()+ "</s>").ids, 150)
    #print(src_indexes)
    #torch.tensor(pad_sequences(x,self.max_len_src), dtype=torch.long)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    #src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    #print(src_mask)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [0]
    
    for i in range(max_len):

        #trg_tensor = torch.LongTensor(pad_sequences(trg_indexes, 50)).unsqueeze(0).to(device)
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        #print(trg_tensor)
        trg_mask = model.make_trg_mask(trg_tensor)
        #print(trg_mask)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        #print(output[0][-1].argmax())
        #print(pred_token)
        trg_indexes.append(pred_token)

        if pred_token == 2:
            break
    print(trg_indexes)
    trg_tokens = [tgt_tokenizer.decode([i]) for i in trg_indexes]
    
    return trg_tokens, attention

In [ ]:
s  = "public static List < String > read Lines ( File file ) throws IO Exception { return read Lines ( file , Charset . default Charset ( ) ) ; }"
#s = "private void update Rating Choice ( ) { int current = m ch Rating . get Selected Index ( ) ; m ch Rating . remove All Items ( ) ; Faction Record f Rec = ( Faction Record ) m ch Subfaction . get Selected Item ( ) ; if ( f Rec == null ) { f Rec = ( Faction Record ) m ch Faction . get Selected Item ( ) ; } Array List < String > rating Levels = f Rec . get Rating Levels ( ) ; if ( rating Levels . is Empty ( ) ) { rating Levels = f Rec . get Rating Level System ( ) ; } if ( rating Levels . size ( ) > NUM ) { for ( int i = rating Levels . size ( ) - NUM ; i >= NUM ; i -- ) { m ch Rating . add Item ( rating Levels . get ( i ) ) ; } } if ( current < NUM && m ch Rating . get Item Count ( ) > NUM ) { m ch Rating . set Selected Index ( NUM ) ; } else { m ch Rating . set Selected Index ( Math . min ( current , m ch Rating . get Item Count ( ) - NUM ) ) ; } }"
#s = "public static boolean memory Is Low ( ) { return available Memory ( ) * NUM < RUNTIME . total Memory ( ) * NUM ; }"
out, _ = translate_sentence(s, model, 'cuda')
print(out)

src_tokenizer.decode([352])

In [ ]:
%matplotlib inline
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()
    
translation, attention = translate_sentence(s, model, 'cuda')
display_attention(s, translation, attention)

In [ ]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

In [ ]:
print(tgt_tokenizer.decode([225, 2, 225, 1]))
print(tgt_tokenizer.decode(tgt_tokenizer.encode("hello there").ids))
print(tgt_tokenizer.encode("<s> hello there <pad> <pad>").ids)
print(tgt_tokenizer.encode('<pad>').ids)
tgt_tokenizer.decode([10304])

In [ ]:
!pip install tqdm

In [ ]:
!head -3 ../data/ncs_preprocessed_data/train-ncs/code.original_subtoken

In [ ]:
!head -3 ../data/ncs_preprocessed_data/train-ncs/javadoc.original

In [ ]:
??tgt_tokenizer

In [ ]:
x = torch.Tensor.new_full(50,20000, dtype=torch.long)

In [ ]:
src_tokenizer

In [ ]:
torch.tensor.new_full((3, 4), 3.141592)

In [ ]:
import gc
gc.collect()

In [ ]:
!nvidia-smi

In [ ]:
len(tgt_tokenizer.encode("reads the contents of a file line by line to a list of strings using the default encoding for the vm . the file is always closed .").ids)

In [ ]:
dataset = DataLoader(train_dataset, batch_size = 10, num_workers=10,
                     drop_last=True,
                     shuffle=True)

for (src_, trg_) in dataset:
    print(src_[0])
    break

In [ ]:
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B8EI5pQUswtUuiYwiInxCb02d5t1NHKH' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B8EI5pQUswtUuiYwiInxCb02d5t1NHKH" -O ncs_big.7z && rm -rf /tmp/cookies.txt


